In [81]:
from openai import OpenAI
from jobsearch.database import fetch_data_from_postgresql
from jobsearch.params import OPENAI_API_KEY
from tqdm import tqdm

from sklearn.model_selection import train_test_split
from datasets import Dataset, DatasetDict, load_dataset, concatenate_datasets

import numpy as np
import pandas as pd
import json
from datetime import datetime

from jobsearch.utils import request_openai, validate_jsonschema

# pd.options.display.max_colwidth = 100
# pd.options.display.max_rows = 20

### Load data

In [82]:
data = fetch_data_from_postgresql(import_from_cloud=False)

Using LOCAL postgres database ...


### Prepro

In [83]:
df = data.copy()

# remove duplicates and nulls
df = df.drop_duplicates('description')
df = df.dropna(subset='description', axis=0)

# subset by date
df_subset = df[(df.date_time.dt.date >= (pd.to_datetime('2024-04-07').date())) & (df.date_time.dt.date <= (pd.to_datetime('2024-04-11').date()))]

# keep descriptions and ids
description_subset = df_subset['description']
id_subset = df_subset['id']

# convert to list
data_to_request = description_subset.copy().to_list()
ids = id_subset.copy().to_list()

len(data_to_request)

841

#### Get instructions

In [85]:
key = 'poste'

# INSTRUCTIONS
instruction_path = f"../../data/prompts/{key}_extraction_instructions.txt" 

with open(instruction_path, "r") as f:
    instructions = f.read()

# JSON SCHEMA FOR OUTPUT VALIDATION
json_schema_path = f'../../data/json_schema/{key}_json_schema.txt'

with open(json_schema_path, "r") as f:
    json_schema = f.read()


### Request OpenAI API

In [86]:
openai_outputs = [request_openai(instructions=instructions, input=datum, json_output=True, print_output=True, model='gpt-3.5-turbo-1106') for datum in tqdm(data_to_request)]

  0%|          | 1/841 [00:06<1:33:30,  6.68s/it]

{
  "poste": {
    "type_contrat": "CDI",
    "teletravail_possible": true,
    "experience_pro_requise": 2,
    "diplomes": ["pas de diplome"],
    "spe_diplomes": [null],
    "langues": ["Anglais"],
    "stack_technique": ["Vue.js", "IA", "Machine Learning"]
  }
}


  0%|          | 2/841 [00:08<50:42,  3.63s/it]  

{
  "poste": {
    "type_contrat": "CDI",
    "teletravail_possible": null,
    "experience_pro_requise": 5,
    "diplomes": ["Bac+5"],
    "spe_diplomes": ["mathématiques appliquées", "statistiques", "Machine Learning"],
    "langues": ["Anglais"],
    "stack_technique": ["Python", "R", "SAS"]
  }
}


  0%|          | 3/841 [00:11<45:56,  3.29s/it]

...

{
  "poste": {
    "type_contrat": "CDI",
    "teletravail_possible": false,
    "experience_pro_requise": 1,
    "diplomes": ["Master", null],
    "spe_diplomes": ["Ingénierie", null],
    "langues": ["Anglais", "Francais", null],
    "stack_technique": ["Python", "Tensorflow", "Unity", "Pytorch", "Google Cloud"]
  }
}


100%|█████████▉| 840/841 [35:56<00:01,  1.94s/it]

{
  "poste": {
    "type_contrat": "CDI",
    "teletravail_possible": false,
    "experience_pro_requise": 5,
    "diplomes": ["pas de diplome"],
    "spe_diplomes": [null],
    "langues": ["Anglais", "Francais"],
    "stack_technique": ["Machine Learning", "AI", "Data Science", "Agile methodologies"]
  }
}


100%|██████████| 841/841 [35:58<00:00,  2.57s/it]

{
  "poste": {
    "type_contrat": "CDI",
    "teletravail_possible": true,
    "experience_pro_requise": 7,
    "diplomes": ["Doctorat"],
    "spe_diplomes": ["Data Science", "Intelligence Artificielle"],
    "langues": ["Francais", "Anglais"],
    "stack_technique": ["Python", "R", "SQL", "pySpark", "Spark", "NoSQL"]
  }
}


### jsonschema validity

In [87]:
validity_list = [validate_jsonschema(schema=json_schema, json_string=output) for output in openai_outputs]

In [88]:
df = pd.DataFrame(data=[ids, data_to_request, openai_outputs, validity_list]).T.rename(columns={0:'id', 1:f'description', 2:f'{key}', 3:'jsonschema_validity'})

print(df.shape) # 841, 4

(841, 4)


In [89]:
file_path = f'../../data/fine-tuning_data/{key}_RAW_ft_data.csv'
df.to_csv(file_path, header=True, index=False)

In [100]:

# path = "../data/fine-tuning_data/json_finetuning_data_alpaca_format.jsonl"
# ft_data = []
# with open(path, "r") as f:
#     for line in f:
#         json_str = json.loads(line)
#         json_str['instruction'] = instructions
#         ft_data.append(json_str)
        
# ft_data
#     for instruction, (id, input_text, output_json_string) in list(zip([instructions]*len(filtered2), filtered2)):

#             alpaca_format = {
                
#                 "instruction": instruction,  # Truncating prompt for brevity
#                 "input": input_text,    # Truncating input for brevity
#                 "output": output_json_string #str(output).replace("'", '"')
#             }
            
#             print(alpaca_format)
            
#             # Serialize the entire object to a JSON string and append a newline
#             jsonl_string = json.dumps(alpaca_format, ensure_ascii=False) + '\n'
            
#             # Write the JSON string directly to the file
#             f.write(jsonl_string)

# read and split
# tiger_llama =  pd.read_json('../data/fine-tuning_data/extracted_salaries_ft_data.jsonl', lines=True)
# train_tiger, test_tiger = train_test_split(
#     tiger_llama, test_size=0.15, random_state=42, shuffle=True
# )

# # push to hub
# ft_data_df_hf = Dataset.from_pandas(ft_data_df)
# ft_data_df_hf
# ds = DatasetDict()
# ds["train"] = train_tiger
# ds["test"] = test_tiger

# dataset_name = "axel-rda/salary_extraction_ft_dataset"
# ds.push_to_hub(dataset_name, branch="main", private=True, )

In [44]:
# from jobsearch.ml_logic.lang_classifier import detect_language
# from sklearn.metrics import classification_report, accuracy_score
# from datasets import load_dataset

# ### evaluate lang detection model accuracy

# # dataset = load_dataset("papluca/language-identification", split="test")

# # test_data = dataset['text'][0:100]

# true_labels = df_lang
# predicted_labels = detect_language(description_list.tolist())

# print(classification_report(true_labels, predicted_labels))
# print(f"Accuracy: {accuracy_score(true_labels, predicted_labels)}")